In [1]:

%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


In [2]:

import numpy as np
import matplotlib.pyplot as plt
from fgbuster import CMB, Dust, Synchrotron, MixingMatrix
from fgbuster.observation_helpers import standardize_instrument, get_observation
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import healpy as hp
import numdifftools as nd
import math as m

In [3]:
instr = np.load('/Users/alicepirotta/Desktop/APC/MCMC/instrument_LB_IMOv1.npy', allow_pickle=True).item()
instr_ = {}
instr_['frequency'] = np.array([instr[f]['freq'] for f in instr.keys()])
instr_['depth_p'] = np.array([instr[f]['P_sens'] for f in instr.keys()])
instr_['fwhm'] = np.array([instr[f]['beam'] for f in instr.keys()])
instr_['depth_i'] = instr_['depth_p']/np.sqrt(2)
instrument = standardize_instrument(instr_)


In [4]:
nside = 4
freq_maps = get_observation(instrument, 'd0s0', noise=True, nside=nside)


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:

components= [CMB(), Dust(50.), Synchrotron(50.)]

In [6]:

A = MixingMatrix(*components)
A_ev = A.evaluator(instrument.frequency)
#invN=np.linalg.inv(np.eye(len(instrument.frequency)))
invN = np.diag((hp.nside2resol(nside, arcmin=True) / instrument.depth_p)**2)

LIKELIHOOD WITHOUT GAIN

In [7]:
def spectral_likelihood(x):
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =A_ev(x) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(x,logL)
    if logL != logL:
        return 0.0
    return -logL

In [8]:
x0 =np.array([1.6,30,-7]) 
x1 = np.random.uniform(low=[x0[0] * (1 - 1 / 40), x0[1] * (1 - 1 / 40), x0[2] * (1 - 1 / 40)], high=[x0[0] * (1 + 1 / 40), x0[1] * (1 + 1 / 40), x0[2] * (1 + 1 / 40)], size=(len(x0)))

min_= minimize (spectral_likelihood, x0, method='trust-constr')
print(min_)
print(min_.x)

          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595812176274.74
                x: [ 1.457e+00  2.440e+01 -7.232e-01]
              nit: 62
             nfev: 248
             njev: 62
             nhev: 0
         cg_niter: 161
     cg_stop_cond: 2
             grad: [ 3.106e+08  8.057e+05 -8.677e+07]
  lagrangian_grad: [ 3.106e+08  8.057e+05 -8.677e+07]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 310596254.0298217
 constr_violation: 0
   execution_time: 0.24312615394592285
        tr_radius: 4.4945570909998425e-09
   constr_penalty: 1.0
            niter: 62
[ 1.45674823 24.4016673  -0.72317264]


In [9]:
print(min_.x)

[ 1.45674823 24.4016673  -0.72317264]


In [10]:
# x0= np.array([1.54,20,-3])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =A_ev(min_.x) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)
# ps_CMB =hp.anafast(s[0])
# #ps_CMB =hp.sphtfunc.anafast(s[0])
# plt.loglog(ps_CMB)
# print (ps_CMB.shape)
# plt.xlabel(r'$\ell$')



In [11]:
# x0= np.array([1.54,20,-3])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =A_ev(min_.x) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)

# ps_CMB =hp.anafast(s[0])
# ps_CMB=ps_CMB.T
# ps_CMB=ps_CMB[2:,:]

# ell = np.arange(2,192)

# values = []
# for i in range(len(ps_CMB[0,:])):
#     y = (ell*(ell+1)*ps_CMB[:,i])/(2*m.pi)
#     values.append(y)


# plt.loglog(ell,values[0], label="TT")
# plt.loglog(ell,values[1], label="EE")
# plt.loglog(ell,values[2], label="BB")
# plt.loglog(ell,values[3], label="TE")
# plt.loglog(ell,values[4], label="EB")
# plt.loglog(ell,values[5], label="TB")


# plt.xlabel(r'$\ell$')
# plt.ylabel(r'$\ell$* ($\ell$+1)*C$\ell$/2$\pi$')
# plt.legend()

In [12]:

print(spectral_likelihood(x0))

-31568660090317.15


hessian matrix

In [13]:
H = nd.Hessian(spectral_likelihood)
h = H([1.54, 20, -3])
print(h)




[[9.24011670e+11 2.51498740e+10 3.25418859e+09]
 [2.51498740e+10 6.91880777e+08 7.94323737e+07]
 [3.25418859e+09 7.94323737e+07 4.03067342e+07]]


eigen vector, eigen values

In [14]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v)

LIKELIHOOD WITH GAIN (1 variables, 1 fixed)

In [15]:
def spectral_likelihood(y):
    Bd, T, Bs, a = y
    b = 1
    G = np.diag([a,a,a,a,a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,b,b,b])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [16]:
# x0= np.array([1.54,20,-3,1])
# a, b = 1, 1
# G = np.diag([a,a,a,a,a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,b,b,b])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =G.dot(A_ev(x0)) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)
# Q =hp.sphtfunc.anafast(s[0])
# plt.plot(Q)


In [17]:
x0 =np.array([1.4, 21, -2.5, 1])  
x1 = np.random.uniform(low=[x0[0] * (1 - 1 / 40), x0[1] * (1 - 1 / 40), x0[2] * (1 - 1 / 40),  x0[3] * (1 - 1 / 40)], high=[x0[0] * (1 + 1 / 40), x0[1] * (1 + 1 / 40), x0[2] * (1 + 1 / 40),  x0[3] * (1 + 1 / 40)], size=(len(x0)))

min_= minimize (spectral_likelihood,x1,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)

          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595872093742.54
                x: [ 1.536e+00  2.015e+01 -2.838e+00  9.997e-01]
              nit: 332
             nfev: 1625
             njev: 325
             nhev: 0
         cg_niter: 1223
     cg_stop_cond: 2
             grad: [ 2.049e+06  0.000e+00 -3.602e+06 -5.243e+05]
  lagrangian_grad: [ 2.049e+06  0.000e+00 -3.602e+06 -5.243e+05]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 3601948.0984733747
 constr_violation: 0
   execution_time: 1.2369301319122314
        tr_radius: 5.799969657769938e-19
   constr_penalty: 1.0
            niter: 332
[ 1.53550113 20.14600971 -2.83835738  0.99973593]


In [18]:
print(min_.x)

[ 1.53550113 20.14600971 -2.83835738  0.99973593]


LIKELIHOOD WITH GAIN (2 variables, 1 fixed)

In [19]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c = y
    b = 1
    G = np.diag([a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,c,c,c,c,c,c,c])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [20]:
x0 =np.array([1.4, 21, -2.5, 1, 1])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)

          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595870461259.707
                x: [ 1.512e+00  2.100e+01 -2.497e+00  9.945e-01  9.988e-01]
              nit: 57
             nfev: 312
             njev: 52
             nhev: 0
         cg_niter: 131
     cg_stop_cond: 2
             grad: [ 5.203e+05  1.585e+06 -4.934e+06  1.153e+07  1.442e+07]
  lagrangian_grad: [ 5.203e+05  1.585e+06 -4.934e+06  1.153e+07  1.442e+07]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 14417920.0
 constr_violation: 0
   execution_time: 0.2036750316619873
        tr_radius: 7.124184466999679e-19
   constr_penalty: 1.0
            niter: 57
[ 1.51153378 21.00279006 -2.49729862  0.99450336  0.99883056]


In [21]:
H = nd.Hessian(spectral_likelihood)
h = H([1.54, 20, -3, 1.1, 1.5])
print(h)

[[ 7.57720088e+11  2.14803641e+10  1.46421438e+10  1.04169054e+09
  -1.85703254e+11]
 [ 2.14803641e+10  1.01105446e+09  3.46430056e+08 -2.97710192e+07
  -4.73197455e+09]
 [ 1.46421438e+10  3.46430056e+08 -9.39496088e+08  4.88859506e+09
  -4.88739393e+09]
 [ 1.04169054e+09 -2.97710192e+07  4.88859506e+09  4.83569520e+09
   1.22490672e+10]
 [-1.85703254e+11 -4.73197455e+09 -4.88739393e+09  1.22490672e+10
   1.13341267e+11]]


In [22]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v.T)

LIKELIHOOD WITH GAIN ( 5 variables, 1 fixed)

In [23]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, d, e, f = y
    b = 1
    G = np.diag([a,a,a,a,b,b,b,b,c,c,c,c,d,d,d,d,e,e,e,f,f,f])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [24]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595872305698.566
                x: [ 1.540e+00  1.999e+01 -2.950e+00  9.979e-01  1.000e+00
                     1.000e+00  9.999e-01  9.998e-01]
              nit: 917
             nfev: 8226
             njev: 914
             nhev: 0
         cg_niter: 6570
     cg_stop_cond: 2
             grad: [ 7.489e+06  3.017e+05 -5.065e+06  8.389e+06  9.436e+06
                     7.340e+06  1.756e+07  9.437e+06]
  lagrangian_grad: [ 7.489e+06  3.017e+05 -5.065e+06  8.389e+06  9.436e+06
                     7.340e+06  1.756e+07  9.437e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 17563648.0
 constr_violation: 0
   execution_time: 4.792257070541382
        tr_radius: 6.458044058344786e-19
   constr_penalty: 1.0
   

LIKELIHOOD WITH GAIN (8 variables,1 fixed)

In [25]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, d, e, f, g, h, i  = y
    b = 1
    G = np.diag([a, a, a, b, b, c, c, c, d, d, e, e, f, f, f, g, g, h, h, i, i, i ])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [26]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.5, 1.5, 0.8])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595871992631.16
                x: [ 1.533e+00  2.021e+01 -2.765e+00  9.952e-01  1.003e+00
                     1.005e+00  1.006e+00  1.006e+00  1.005e+00  1.005e+00
                     1.005e+00]
              nit: 940
             nfev: 11232
             njev: 936
             nhev: 0
         cg_niter: 8996
     cg_stop_cond: 2
             grad: [ 4.274e+06  7.784e+04 -2.844e+05  6.029e+06  5.226e+06
                     6.260e+06  1.225e+07  9.383e+06  8.865e+06  9.388e+06
                     4.694e+06]
  lagrangian_grad: [ 4.274e+06  7.784e+04 -2.844e+05  6.029e+06  5.226e+06
                     6.260e+06  1.225e+07  9.383e+06  8.865e+06  9.388e+06
                     4.694e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_

LIKELIHOOD WITH GAIN (11 variables, 1 fixed)

In [27]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, g, h, i, l, m, n  = y
    f = 1
    G = np.diag([a, a, b, b, c, c, d, d, e, e, g, g, h, h, i, i, l, l, m, m, n, n ])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [28]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.5, 1.5, 0.8, 1.2,0.6,0.3])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: The maximum number of function evaluations is exceeded.
          success: False
           status: 0
              fun: -31595871274218.504
                x: [ 1.513e+00  2.091e+01 ...  1.104e+00  1.104e+00]
              nit: 1000
             nfev: 15000
             njev: 1000
             nhev: 0
         cg_niter: 11667
     cg_stop_cond: 1
             grad: [ 4.660e+07  1.567e+06 ... -4.085e+07  1.588e+08]
  lagrangian_grad: [ 4.660e+07  1.567e+06 ... -4.085e+07  1.588e+08]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 158799198.05373114
 constr_violation: 0
   execution_time: 8.270709037780762
        tr_radius: 0.04670651897798104
   constr_penalty: 1.0
            niter: 1000
[ 1.51331266 20.91075992 -2.60239391  1.11518689  1.10061682  1.10543351
  1.10349248  1.10456521  1.10403772  1.10435403  1.10343482  

LIKELIHOOD WITH GAIN (21 variables, 1 fixed)

In [58]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, n, o, p, q, r, t, u, v, w, z = y
    s = 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [103]:
#np.random.seed(10)
x0 =np.array([1.54,20,-3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
pos = np.random.uniform(low=x0 * (1 - 1 / 4), high= x0 * (1 + 1 / 4))
min_= minimize (spectral_likelihood,pos,method='TNC', options={'maxfun':1000000000},tol= 1e-18 ) 
print(min_)
print(min_.x)


 message: Converged (|f_n-f_(n-1)| ~= 0)
 success: True
  status: 1
     fun: -31595229577667.793
       x: [ 1.496e+00  1.591e+01 ...  1.062e+00  1.098e+00]
     nit: 204
     jac: [ 1.715e+09  7.148e+07 ... -2.227e+09  5.626e+09]
    nfev: 33975
[ 1.49583392 15.91053331 -2.40126294  1.14164614  0.89622046  1.0443041
  1.18274173  1.18440488  1.04662422  1.05942349  0.83149527  0.94204974
  1.0655743   0.9746684   0.98753008  1.00921561  0.99282629  0.98758486
  0.98953241  0.99834829  1.01302388  1.03272045  1.0618278   1.09763903]


In [31]:

Fisher = nd.Hessian(spectral_likelihood)
F = Fisher(min_.x)
print(F)



[[ 8.56523166e+11  2.39258317e+10  5.39302169e+09 -7.57951330e+09
  -2.71526251e+09 -9.82644742e+08  1.92763169e+09  7.64368175e+08
   7.62380784e+09  5.03126109e+09  1.79665619e+10  3.27640740e+09
   3.07489141e+10  5.97265188e+10  4.19659069e+10  4.57130684e+10
   1.08708094e+11  5.38737921e+10 -1.10906602e+11 -1.74475361e+11
  -6.26749452e+11 -1.09149188e+12 -5.94660922e+11  2.46830424e+12]
 [ 2.39258317e+10  6.74582304e+08  1.36061415e+08 -1.85050875e+08
  -6.93574047e+07 -3.25376782e+07  3.81290170e+07  1.51193801e+07
   1.75968229e+08  1.16128599e+08  4.34926483e+08  7.93138056e+07
   7.69368999e+08  1.58534672e+09  1.28774402e+09  1.14378732e+09
   2.88548578e+09  1.65314320e+09 -2.19564826e+09 -4.34873514e+09
  -1.70474673e+10 -3.21966001e+10 -2.19925965e+10  7.33164677e+10]
 [ 5.39302169e+09  1.36061415e+08  1.01873305e+08 -2.20594698e+08
  -1.59063155e+07  9.38166664e+07  1.42283657e+08  5.64195704e+07
   2.42208713e+08  1.59843843e+08  3.46896593e+08  6.32602354e+07
   3.807

In [32]:
Nbd=64 
sqr_Nbd=np.sqrt(Nbd)
Ntd=8
sqr_Ntd= np.sqrt(Ntd)
Nbs=4
sqr_Nbs=np.sqrt(Nbs)
g=1
Npatch= np.matrix([[Nbd,np.sqrt(Nbd*Ntd),np.sqrt(Nbd*Nbs),sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd], [np.sqrt(Ntd*Nbd),Ntd,np.sqrt(Ntd*Nbs),sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd], 
                [np.sqrt(Nbs*Nbd),np.sqrt(Nbs*Ntd),Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Ntd,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs], [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g] ])

In [33]:
# print(Npatch)

In [34]:
# F1= F*Npatch

# print(F1)


In [35]:

# sqrt_inv_F1= np.sqrt(np.linalg.inv(F1))
# print(sqrt_inv_F1)


In [36]:

# sqrt_inv_F= np.sqrt(np.linalg.inv(F))
# print(sqrt_inv_F)

In [37]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v)

LIKELIHOOD WITH GAIN (11 variables, the low frequency band varies)

In [38]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, = y
    n, o, p, q, r, s, t, u, v, w, z = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [39]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595870671419.676
                x: [ 1.513e+00  2.100e+01 ...  1.001e+00  1.001e+00]
              nit: 249
             nfev: 3645
             njev: 243
             nhev: 0
         cg_niter: 1822
     cg_stop_cond: 2
             grad: [ 3.812e+06  3.396e+06 ...  1.022e+07  5.502e+06]
  lagrangian_grad: [ 3.812e+06  3.396e+06 ...  1.022e+07  5.502e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 12058624.0
 constr_violation: 0
   execution_time: 1.6967082023620605
        tr_radius: 6.339825214901231e-19
   constr_penalty: 1.0
            niter: 249
[ 1.51270312 20.99669137 -2.50064301  1.05847935  1.010671    0.99910888
  0.99839679  0.998596    0.99959547  0.99954597  1.0002788   1.00009344
  1.00074158 

LIKELIHOOD WITH GAIN ( 11 variables, every two gain)

In [40]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, e, g, i, m, o, q, s, u, w = y
    b, d, f, h, l, n, p, r, t, v, z = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [41]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595871403374.305
                x: [ 1.513e+00  2.100e+01 ...  9.998e-01  1.000e+00]
              nit: 628
             nfev: 9360
             njev: 624
             nhev: 0
         cg_niter: 5961
     cg_stop_cond: 2
             grad: [ 5.545e+06  1.011e+06 ...  3.932e+06  1.337e+07]
  lagrangian_grad: [ 5.545e+06  1.011e+06 ...  3.932e+06  1.337e+07]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 13365251.10885021
 constr_violation: 0
   execution_time: 4.94277286529541
        tr_radius: 7.83544462320563e-19
   constr_penalty: 1.0
            niter: 628
[ 1.51268998 20.99777545 -2.50928041  1.06481433  1.00120831  1.00033016
  0.99998332  1.00019535  1.00044437  1.00054662  1.00008276  0.99965814
  0.99982

LIKELIHOOD WITH GAIN ( 11 variables, the middle and the high frequency band varies )

In [42]:
def spectral_likelihood(y):
    Bd, T, Bs, n, o, p, q, r, s, t, u, v, w, z = y
    a, b, c, d, e, f, g, h, i, l, m = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [43]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: The maximum number of function evaluations is exceeded.
          success: False
           status: 0
              fun: -31595869431310.93
                x: [ 1.122e+00  2.099e+01 ...  1.260e+00  1.341e+00]
              nit: 1000
             nfev: 14955
             njev: 997
             nhev: 0
         cg_niter: 9303
     cg_stop_cond: 2
             grad: [-1.378e+07 -2.247e+05 ...  1.103e+07 -6.840e+06]
  lagrangian_grad: [-1.378e+07 -2.247e+05 ...  1.103e+07 -6.840e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 13782606.732853238
 constr_violation: 0
   execution_time: 8.024792909622192
        tr_radius: 3.8746888499294035e-06
   constr_penalty: 1.0
            niter: 1000
[ 1.12217495 20.99497715 -2.50041212  1.01683306  0.9951743   1.00168449
  1.01681855  1.04232321  1.07571876  1.07571816  1.12611342  

LIKELIHOOD WITH GAIN ( 5 variables, the high frequency band varies)

In [56]:
def spectral_likelihood(y):
    Bd, T, Bs, t, u, v, w, z  = y
    a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [57]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', options={'maxiter': 1000000, 'disp': True} , tol= 1e-18)
print(min_)
print(min_.x)


`xtol` termination condition is satisfied.
Number of iterations: 323, function evaluations: 2871, CG iterations: 1830, optimality: 9.70e+06, constraint violation: 0.00e+00, execution time:  1.6 s.
          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595872194946.27
                x: [ 1.518e+00  2.099e+01 -2.837e+00  1.000e+00  1.000e+00
                     1.000e+00  9.997e-01  9.987e-01]
              nit: 323
             nfev: 2871
             njev: 319
             nhev: 0
         cg_niter: 1830
     cg_stop_cond: 2
             grad: [ 4.145e+06  1.874e+05 -2.033e+06 -5.243e+05  6.028e+06
                     9.698e+06  1.835e+06  5.767e+06]
  lagrangian_grad: [ 4.145e+06  1.874e+05 -2.033e+06 -5.243e+05  6.028e+06
                     9.698e+06  1.835e+06  5.767e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
   

In [46]:
# def spectral_likelihood(i):
#     Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, n, o, p, q, r, s, t, u, v, w, z = i
#     h = 1
#     G = np.diag([ a, b, c, d, e, f, g, i, h, l, m, n, o, p, q, r, s, t, u, v, w, z])
#     invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
#     A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
#     logL = 0
#     AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
#     AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
#     logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
#     #print(i,logL)
#     if logL != logL:
#         return 0.0
#     return -logL

In [47]:
# tol_ = 1e-18
# x0 =np.array([1.3, 15, -2.5, 0.5, 1.2, 0.1, 2, 0.2, 0.9, 1.1, 0.3, 0.9, 1.4, 1.2, 0.4, 0.8, 0.9, 1.2, 1.3, 0.7, 0.2, 1.9, 0.9, 1.3, 1.4]) 
# min_= minimize (spectral_likelihood,x0,method='trust-constr')
# print(min_)
# print(min_.x)


In [48]:
# g1 = np.linspace(0.5,1.5,500)
# result= []
# for i in g1:
#         result.append(spectral_likelihood(np.array([1.26466502, 17.99580668 , -2.65181336 , 1.02481412 , 1.13362434 , 1.13395641,
#   1.10611243 , 1.10360917 ,i  , 1.06613316 , 1.03197411 , 1.03194025,
#   1.00891511,  0.98736361 , 0.98027652,  1.00900499 , 0.98735861  ,0.98027463,
#   0.9850154  , 1.00000578,  1.02959818 , 1.06909778 , 1.12300709 , 1.18561135])))
# print(result)

# plt.plot(g1,result,'o')
# plt.title(r'Likelihood varying g1')
# plt.xlabel(r'slope index, g1')
# plt.ylabel('Likelihood value')

In [49]:
# Bd = np.linspace(1.4,1.6,20)
# g3 = np.linspace(0.8,1.2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,-3,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, g3, result.T,100)
# ax.contour(Bd, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [50]:
# T = np.linspace(15,25,20)
# g3 = np.linspace(0.5,1.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(T):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([1.54,val_i,-3,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(T, g3, result.T,100)
# ax.contour(T, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'T')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [51]:
# Bs = np.linspace(-3.2,-2.8,20)
# g3 = np.linspace(0.8,1.2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bs):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([1.54,20,val_i,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bs, g3, result.T,100)
# ax.contour(Bs, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_s$')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
# plt.show()

In [52]:
# Bd = np.linspace(1.0,2.5,20)
# Bs = np.linspace(-4,-2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,val_j]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, Bs, result.T,100)
# ax.contour(Bd, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'$\beta_s$')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()


In [53]:
# Bd = np.linspace(2.6,3,20)
# Bs = np.linspace(-0.5,0.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,val_j]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, Bs, result.T,100)
# ax.contour(Bd, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'$\beta_s$')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [54]:
# Bd = np.linspace(1.52,1.56,20)
# T = np.linspace(19.5,20.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(T):
#         result[i,j] = spectral_likelihood(np.array([val_i,val_j,-3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, T, result.T,100)
# ax.contour(Bd, T, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'T')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [55]:
# T = np.linspace(19.9,20.1,20)
# Bs = np.linspace(-3.1,-2.9,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(T):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([1.54,val_i,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(T, Bs, result.T,100)
# ax.contour(T, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'T')
# ax.set_ylabel(r'Betas')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()